<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Investigaci-n-de-operaciones/blob/main/Tarea2IO/Metodo-Grafico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#Funcion

In [ ]:
class Funcion:
  MENOR_IGUAL=0
  MAYOR_IGUAL=1
  MAYOR=2
  MENOR=3
  def __init__(self,coef:float,cort:float, tipo:int=MENOR):
    self.coef=coef
    self.cort=cort
    self.tipo=tipo
  def funcion(coef1,coef2,cort1,cort2, tipo=MENOR):
    cort=(cort2-cort1)/coef1
    coef=coef2/coef1
    if coef1<0:
      if tipo==Funcion.MENOR or tipo ==Funcion.MAYOR:
        tipo=(Funcion.MAYOR if tipo==Funcion.MENOR else Funcion.MENOR)
      else:
        tipo=(Funcion.MAYOR_IGUAL if tipo==Funcion.MENOR_IGUAL else Funcion.MENOR_IGUAL)
    return Funcion(coef,cort,tipo)
  def corte(self):
    return ((0,self.cort),((-self.cort)/self.coef,0))
  def calc(self,x):
    return self.coef*x+self.cort
  def condicion(self,x,y):
    return [lambda a,b: a<=b,lambda a,b: a>=b, lambda a,b:a>b, lambda a,b: a<b][self.tipo](y,self.calc(x))
  def __add__(self,other):
    if self.tipo!=other.tipo:
      return None
    return Funcion(self.coef+other.coef,self.cort+other.cort,self.tipo)
  def __sub__(self,other):
    if self.tipo!=other.tipo:
      return None
    return Funcion(self.coef-other.coef,self.cort-other.cort,self.tipo)
  def __repr__(self):
    return "{}x+({}) {} y".format(self.coef,self.cort, ["<=",">=",">","<"][self.tipo]) 
  def interseccion(self, other):
    p=(self-other).corte()[1][0]
    return p,self.calc(p)


###Verificación

In [ ]:
Funcion(-2,7).corte()

In [ ]:
Funcion(-2,7).interseccion(Funcion(5,-10))

In [ ]:
x=np.linspace(-10,10,100)
plt.plot(x,-2*x+7)
plt.plot(x,5*x-10)
plt.scatter(2.4285714285714284, 2.1428571428571432)

#Solver

In [ ]:
class Solver:
  def __init__(self, condiciones,positivas=True):
    self.condiciones=condiciones
    self.positivas=positivas
  def __puntos(self):
    k=self.condiciones
    p=[]
    for i in range(len(k)):
      for j in range(i+1,len(k)):
        p.append(k[i].interseccion(k[j]))
    if self.positivas:
      for i in range(len(k)):
          w=k[i].corte()
          p.append(w[0])
          p.append(w[1])
    return p
  def solver(self):
    p=self.__puntos()
    s=p[:]
    for i in self.condiciones:
      for j in range(len(p)):
        if (not i.condicion(p[j][0],p[j][1])) or (p[j][1]<0 or p[j][0]<0 if self.positivas else False):
          if p[j] in s:
            s.remove(p[j])
    return s

In [ ]:
w=Solver([Funcion(5,2,Funcion.MENOR_IGUAL), Funcion(-2,10,Funcion.MENOR_IGUAL)])
w.solver()

In [ ]:
x=np.linspace(-10,10,10)
plt.plot(x,5*x+2)
plt.plot(x,-2*x+10)
y=np.array(w.solver()).T

plt.scatter(y[0],y[1])
plt.grid()
plt.show()